In [ ]:
"""
【scrapy的使用】


创建scrapy 项目
scrapy startproject project_name 

创建爬虫
scrapy genspider spider_name 允许爬取的范围

运行爬虫 (进入到爬虫所有目录)
scrapy crawl spider_name


使用pipeline
1、实现pipelines.py中的 process_item 方法

2、去配置文件中开启pipeline
ITEM_PIPELINES = {
    'demo01.pipelines.Demo01Pipeline': 300, #数值越大数先执行
}


使用logging
去配置文件中配置

LOG_LEVEL = "WARNING"
LOG_FILE = "./file.log"	#保存路径，设置后不会再在终端中显示日志，需要使用logger


日志的级别有

CRITICAL - 严重错误
ERROR - 一般错误
WARNING - 警告信息
INFO - 一般信息
DEBUG - 调试信息  默认



import logging

logging.warning('your message') 输出格式: 2019-5-20 [root] WARNING:your message，不知道在哪个文件

logger = loggin.getLogger(__name__)
logger.warning('your message') 输出格式: 2019-5-20 [your file] WARNING:your message

# logging模块的设置可以通过loggin.baseConfig()来完成，百度一下参数即可


关于scrapy中实现翻页思路：
1、找到下一页的地址
2、构造一人下一页url地址的request请求传给调度器

next_url = response.xpath('pettern')
while len(next_url) > 0:
    yield scrapy.Request(next_url,callback=self.parse)





"""